## This notebook generates P3HT trajectories using flowerMD(HOOMD), mbuild, Espaloma, and Grits(CG)

In [2]:
import mbuild as mb
from flowermd.base import Molecule, Pack
from grits import CG_System
import unyt
from flowermd.library import FF_from_file

/Users/stephaniemccallum/miniforge3/envs/flowermd-dev/lib/python3.12/site-packages/gmso/core/element.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


In [2]:
def espaloma_mol(file_path):
     mol = mb.load(file_path)
     for p in mol.particles():
           p.name = f"_{p.name}"
     return mol

In [3]:
esp_mol = espaloma_mol("p3ht20.mol2")
p3ht = Molecule(num_mols=1, compound=esp_mol)
system = Pack(molecules=p3ht, density=0.5 * unyt.g/unyt.cm**3)
p3ht_ff = FF_from_file(forcefield_files="P3HT.xml")
system.apply_forcefield(r_cut=2.5, force_field=p3ht_ff, auto_scale=True, remove_hydrogens=False)
system.to_gsd('init_p3ht.gsd')

/Users/stephaniemccallum/miniforge3/envs/flowermd-dev/lib/python3.12/site-packages/gmso/formats/mol2.py:80: UserWarning: The record type indicator Meta is not supported. Skipping current section and moving to the next RTI header.
  warnings.warn(
/Users/stephaniemccallum/miniforge3/envs/flowermd-dev/lib/python3.12/site-packages/gmso/formats/mol2.py:80: UserWarning: The record type indicator @<TRIPOS>SUBSTRUCTURE is not supported. Skipping current section and moving to the next RTI header.
  warnings.warn(
/Users/stephaniemccallum/miniforge3/envs/flowermd-dev/lib/python3.12/site-packages/mbuild/compound.py:546: UserWarning: Some particle of <Compound 502 particles, 521 bonds, non-periodic, id: 6118297376> does not have a charge.They will not be accounted for during this calculation.
  warn(
/Users/stephaniemccallum/miniforge3/envs/flowermd-dev/lib/python3.12/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _H5
  warning

In [4]:
from flowermd.base import Simulation
gsd_path='aa_p3ht_20mer.gsd'
sim = Simulation.from_system(
    system=system,
    dt=0.001,
    gsd_write_freq=1000,
    gsd_file_name=gsd_path,
    log_write_freq=1000,
    log_file_name='log.txt')

sim.run_NVT(n_steps=5e5, kT=2.0, tau_kt=10*sim.dt)
sim.flush_writers()

*Warning*: charge.pppm: system is not neutral and unscreened interactions are calculated, the net charge is -3.64009
*Warning*: charge.pppm: RMS error of 0.196568 is probably too high! 0.196527 0.196568


Initializing simulation state from a gsd.hoomd.Frame.
Step 1000 of 500000; TPS: 463.62; ETA: 17.9 minutes
Step 2000 of 500000; TPS: 462.47; ETA: 17.9 minutes
Step 3000 of 500000; TPS: 462.5; ETA: 17.9 minutes
Step 4000 of 500000; TPS: 462.6; ETA: 17.9 minutes
Step 5000 of 500000; TPS: 463.77; ETA: 17.8 minutes
Step 6000 of 500000; TPS: 464.86; ETA: 17.7 minutes
Step 7000 of 500000; TPS: 464.89; ETA: 17.7 minutes
Step 8000 of 500000; TPS: 463.9; ETA: 17.7 minutes
Step 9000 of 500000; TPS: 463.08; ETA: 17.7 minutes
Step 10000 of 500000; TPS: 462.63; ETA: 17.7 minutes
Step 11000 of 500000; TPS: 462.27; ETA: 17.6 minutes
Step 12000 of 500000; TPS: 461.41; ETA: 17.6 minutes
Step 13000 of 500000; TPS: 460.99; ETA: 17.6 minutes
Step 14000 of 500000; TPS: 461.03; ETA: 17.6 minutes
Step 15000 of 500000; TPS: 461.22; ETA: 17.5 minutes
Step 16000 of 500000; TPS: 462.08; ETA: 17.5 minutes
Step 17000 of 500000; TPS: 461.98; ETA: 17.4 minutes
Step 18000 of 500000; TPS: 462.34; ETA: 17.4 minutes
Step

In [6]:
from cmeutils.sampling import is_equilibrated
import numpy as np
shrink_cut = 200
log = np.genfromtxt('log.txt', names=True)
pe = log["mdcomputeThermodynamicQuantitiespotential_energy"]
# equil: bool (True or False), t0: int (starting data point that you can sample from), neff: number of data points in equilibrated region
print(len(pe[shrink_cut:]))
equil, t0, g, neff = is_equilibrated(data=pe[shrink_cut:], threshold_neff=100) 
if equil:
    print("True")
else:
    # If it's not equilibrated, job.doc.equilibrated stays false, and the "run-longer" operation in project.py will run next time we submit this job
    print("False")

301
True


In [1]:
thiophene = "c1cscc1"
side_chain= "CCC"

In [3]:
import ele
element_dict = {("C0"): ele.element_from_symbol("C"),
                ("C1"): ele.element_from_symbol("C"),
                ("H3"): ele.element_from_symbol("H"),
                ("H4"): ele.element_from_symbol("H"),
                ("H5"): ele.element_from_symbol("H"),
                ("S2"):ele.element_from_symbol("S")}

cg_system = CG_System(gsdfile='aa_p3ht_50mer.gsd',
                      beads={"A":thiophene,"B":side_chain},
                      conversion_dict=element_dict,
                      add_hydrogens=True,
                     )

Added 0 hydrogens.


In [4]:
cg_gsd = "cg_p3ht_50mer.gsd"
cg_system.save(cg_gsd)